<a href="https://colab.research.google.com/github/venkatesh-manikumar/Datascience-Assignments/blob/main/Predict_The_Flight_Ticket_Price_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
from google.colab import drive
##drive.mount('/content/drive')

In [230]:
import pandas as pd
import holidays
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

In [231]:
train_data = pd.read_excel('/content/drive/MyDrive/DataScience/Flight_price_prediction/Data_Train.xlsx')
train_data.shape

(10683, 11)

In [232]:
train_data.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

Missing data stats

In [233]:
train_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [234]:
train_data[train_data['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [235]:
train_data[train_data['Route'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [236]:
train_data.dropna(inplace=True)

Feature Engineering

In [237]:
#train_data['Total_Stops'] = train_data['Total_Stops'].fillna('1 stop')
train_data['Total_Stops'] = train_data['Total_Stops'].replace('non-stop','0 stop')
train_data['Total_Stops'] = train_data['Total_Stops'].str.split(' ').str[0].astype(int)

In [238]:
train_data['Date'] = train_data['Date_of_Journey'].str.split('/').str[0].astype(int)
train_data['Month'] = train_data['Date_of_Journey'].str.split('/').str[1].astype(int)
train_data['Year'] = train_data['Date_of_Journey'].str.split('/').str[2].astype(int)

In [239]:
train_data['Arrival_Time'] = train_data['Arrival_Time'].str.split(' ').str[0]
train_data['Arrival_Hour'] = train_data['Arrival_Time'].str.split(':').str[0].astype('int')
train_data['Arrival_Minute'] = train_data['Arrival_Time'].str.split(':').str[1].astype('int')
train_data['Departure_Hour'] = train_data['Dep_Time'].str.split(':').str[0].astype('int')
train_data['Departure_Minute'] = train_data['Dep_Time'].str.split(':').str[1].astype('int')

train_data = train_data.drop(['Arrival_Time'],axis=1)
train_data = train_data.drop(['Dep_Time'],axis=1)


train_data['Duration_Hour'] = train_data['Duration'].str.split(' ').str[0]
train_data['Duration_Minute'] = train_data['Duration'].str.split(' ').str[1]
train_data['Duration_Hour']= train_data['Duration_Hour'].str.split('h').str[0]
train_data['Duration_Minute'] = train_data['Duration_Minute'].str.split('m').str[0]

train_data['Duration_Minute'] = train_data['Duration_Minute'].fillna(0).astype('int')
train_data['Duration_Hour'] = train_data['Duration_Hour'].fillna(0)

In [240]:
# df.assign(Percentage = lambda x: (x['Total_Marks'] /500 * 100))
# df = df.apply(lambda x: np.square(x) if x.name == 'd' else x, axis=1)
# train_data['dayofweek'] =train_data['Journey_date'].apply(lambda x:x.dayofweek)

In [241]:
train_data['Journey_date'] = pd.to_datetime(train_data['Date_of_Journey'],dayfirst=True)
#train_data['Day'] =train_data['Journey_date'].apply(lambda x:x.day_name())
train_data['Day_of_week'] =train_data['Journey_date'].apply(lambda x:x.isoweekday())

train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute,Duration_Hour,Duration_Minute,Journey_date,Day_of_week
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,2h 50m,0,No info,3897,24,3,2019,1,10,22,20,2,50,2019-03-24,7
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2,No info,7662,1,5,2019,13,15,5,50,7,25,2019-05-01,3
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2,No info,13882,9,6,2019,4,25,9,25,19,0,2019-06-09,7
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1,No info,6218,12,5,2019,23,30,18,5,5,25,2019-05-12,7
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1,No info,13302,1,3,2019,21,35,16,50,4,45,2019-03-01,5


In [242]:
India_holidays = holidays.India()
train_data['isHoliday'] =train_data['Journey_date'].apply(lambda x: 1 if x in India_holidays else 0)
train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,...,Year,Arrival_Hour,Arrival_Minute,Departure_Hour,Departure_Minute,Duration_Hour,Duration_Minute,Journey_date,Day_of_week,isHoliday
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,2h 50m,0,No info,3897,24,...,2019,1,10,22,20,2,50,2019-03-24,7,0
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2,No info,7662,1,...,2019,13,15,5,50,7,25,2019-05-01,3,1
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2,No info,13882,9,...,2019,4,25,9,25,19,0,2019-06-09,7,1
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1,No info,6218,12,...,2019,23,30,18,5,5,25,2019-05-12,7,0
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1,No info,13302,1,...,2019,21,35,16,50,4,45,2019-03-01,5,0


In [243]:
for date,event in India_holidays.items():
  print(date,event)

2019-01-14 Makar Sankranti / Pongal
2019-01-26 Republic Day
2019-08-15 Independence Day
2019-10-02 Gandhi Jayanti
2019-05-01 Labour Day
2019-10-27 Diwali
2019-03-21 Holi
2019-09-09 Day of Ashura* (*estimated)
2019-11-09 Mawlid* (*estimated)
2019-06-04 Eid ul-Fitr* (*estimated)
2019-06-05 Eid ul-Fitr* (*estimated)
2019-08-11 Eid al-Adha* (*estimated)
2019-08-12 Eid al-Adha* (*estimated)
2019-04-14 Palm Sunday
2019-04-19 Good Friday
2019-04-21 Easter Sunday
2019-06-09 Feast of Pentecost
2019-12-25 Christmas Day


In [244]:
train_data.isnull().sum()

Airline             0
Date_of_Journey     0
Source              0
Destination         0
Route               0
Duration            0
Total_Stops         0
Additional_Info     0
Price               0
Date                0
Month               0
Year                0
Arrival_Hour        0
Arrival_Minute      0
Departure_Hour      0
Departure_Minute    0
Duration_Hour       0
Duration_Minute     0
Journey_date        0
Day_of_week         0
isHoliday           0
dtype: int64

In [245]:
train_data['Route_1'] = train_data['Route'].str.split('→ ').str[0]
train_data['Route_2'] = train_data['Route'].str.split('→ ').str[1]
train_data['Route_3'] = train_data['Route'].str.split('→ ').str[2]
train_data['Route_4'] = train_data['Route'].str.split('→ ').str[3]
train_data['Route_5'] = train_data['Route'].str.split('→ ').str[4]

train_data['Route_1'].fillna('None',inplace=True)
train_data['Route_2'].fillna('None',inplace=True)
train_data['Route_3'].fillna('None',inplace=True)
train_data['Route_4'].fillna('None',inplace=True)
train_data['Route_5'].fillna('None',inplace=True)

train_data.head()

,Airline,Date_of_Journey,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,...,Duration_Hour,Duration_Minute,Journey_date,Day_of_week,isHoliday,Route_1,Route_2,Route_3,Route_4,Route_5
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,2h 50m,0,No info,3897,24,...,2,50,2019-03-24,7,0,BLR,DEL,None,None,None
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2,No info,7662,1,...,7,25,2019-05-01,3,1,CCU,IXR,BBI,BLR,None
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2,No info,13882,9,...,19,0,2019-06-09,7,1,DEL,LKO,BOM,COK,None
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1,No info,6218,12,...,5,25,2019-05-12,7,0,CCU,NAG,BLR,None,None
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1,No info,13302,1,...,4,45,2019-03-01,5,0,BLR,NAG,DEL,None,None


In [246]:
encoder = LabelEncoder()
train_data['Airline'] = encoder.fit_transform(train_data['Airline'])
train_data['Source'] = encoder.fit_transform(train_data['Source'])
train_data['Destination'] = encoder.fit_transform(train_data['Destination'])
train_data['Route_1'] = encoder.fit_transform(train_data['Route_1'])
train_data['Route_2'] = encoder.fit_transform(train_data['Route_2'])
train_data['Route_3'] = encoder.fit_transform(train_data['Route_3'])
train_data['Route_4'] = encoder.fit_transform(train_data['Route_4'])
train_data['Route_5'] = encoder.fit_transform(train_data['Route_5'])
train_data['Additional_Info'] = encoder.fit_transform(train_data['Additional_Info'])


In [247]:
train_data.drop(['Route'],axis=1,inplace=True)
train_data.drop(['Duration'],axis=1,inplace=True)
train_data.drop(['Date_of_Journey'],axis=1,inplace=True)
train_data.drop(['Journey_date'],axis=1,inplace=True)



In [248]:
train_data['Duration_Minute']

0        50
1        25
2         0
3        25
4        45
         ..
10678    30
10679    35
10680     0
10681    40
10682    20
Name: Duration_Minute, Length: 10682, dtype: int64

In [249]:
lin_reg = LinearRegression()
train_labels = train_data['Price'].copy()
train= train_data.drop('Price',axis=1)
lin_reg.fit(train,train_labels)

ValueError: ignored